# 편의점

# GS25

In [25]:
import requests
from bs4 import BeautifulSoup as BS
import pickle
r = requests.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")

In [26]:
payload = {"pageNum": "1", 
"pageSize": "100", 
"searchShopName": "", 
"searchSido": "", 
"searchGugun": "", 
"searchDong": "", 
"searchType": "", 
"searchTypeService": "0", 
"searchTypeToto": "0", 
"searchTypeCafe25": "0", 
"searchTypeInstant": "0", 
"searchTypeDrug": "0", 
"searchTypeSelf25": "0", 
"searchTypePost": "0", 
"searchTypeATM": "0", 
"searchTypeWithdrawal": "0", 
"searchTypeTaxrefund": "0", 
"searchTypeSmartAtm": "0", 
"searchTypeSelfCookingUtensils": "0", 
"searchTypeDeliveryService": "0", 
"searchTypeParcelService": "0", 
"searchTypePotatoes": "0", 
"searchTypeCardiacDefi": "0", 
"searchTypeFishShapedBun": "0", 
"searchTypeWine25": "0"}

total = []
with requests.Session() as s:
    r = s.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")
    bs = BS(r.text)
    token = bs.find("input", {'name': "CSRFToken"})['value']
    #print(bs.find("input", {'name': "CSRFToken"}))
    target = f"http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={token}"
    for x in range(1, 175):
        if x % 10 == 0: print(x)
        payload['pageNum'] = x
        r2 = s.post(target, data=payload)
        total.append(pd.DataFrame(eval(r2.json())['results']))

print("all done")

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
all done


In [27]:
gs25 = pd.concat(total)

In [28]:
with open("./gs25_prac.pkl", 'wb') as f:
    pickle.dump(gs25, f)

In [22]:
import pickle
import pandas as pd
import numpy as np

In [23]:
with open("./cu.pkl",'rb') as f:
    cu_df = pickle.load(f)

In [24]:
cu_df.shape

(19550, 4)

In [32]:
cu_df[cu_df['store_name'].duplicated()].head(3)

,store_name,store_tel,store_address,store_service
67,고양내유점,,"경기도 고양시 덕양구 유산길 22, (내유동)","[hour, delivery, fried, coffee, cash]"
68,내유무지개점,031,"경기도 고양시 덕양구 유산길 46, (내유동) 1층","[hour, delivery, fried, coffee, cash]"
69,내유희망점,0319625273,"경기도 고양시 덕양구 내유길 81, (내유동) 1층 101,102,103호","[hour, delivery, bakery, fried, coffee, cash]"


In [31]:
cu_df[cu_df.store_name == '고양내유점']

,store_name,store_tel,store_address,store_service
56,고양내유점,,"경기도 고양시 덕양구 유산길 22, (내유동)","[hour, delivery, fried, coffee, cash]"
67,고양내유점,,"경기도 고양시 덕양구 유산길 22, (내유동)","[hour, delivery, fried, coffee, cash]"


In [33]:
cu_df.store_service.loc[0]

['hour', 'delivery', 'fried', 'coffee', 'cash']

In [36]:
unique_cu = cu_df[~cu_df['store_name'].duplicated()].copy()

In [37]:
unique_cu.shape

(17633, 4)

In [38]:
unique_cu.isnull().sum

<bound method DataFrame.sum of        store_name  store_tel  store_address  store_service
0           False      False          False          False
1           False      False          False          False
2           False      False          False          False
3           False      False          False          False
4           False      False          False          False
...           ...        ...            ...            ...
19545       False      False          False          False
19546       False      False          False          False
19547       False      False          False          False
19548       False      False          False          False
19549       False      False          False          False

[17633 rows x 4 columns]>

In [48]:
unique_cu['sido'] = unique_cu.store_address.apply(lambda x : x.split()[0])

In [50]:
len(unique_cu.groupby('sido'))

37

In [53]:
unique_cu.value_counts('sido')

sido
경기도        4770
서울특별시      2859
경상남도       1208
부산광역시      1000
충청남도        922
인천광역시       915
경상북도        905
강원도         824
충청북도        683
제주특별자치도     618
대구광역시       606
전라북도        574
전라남도        515
대전광역시       427
광주광역시       403
울산광역시       370
경남            5
전북특별자치도       4
경북            2
충북            2
대구시           2
서울시           2
전남            2
서울            2
경기            1
광주            1
대구            1
부산광영시         1
부산시           1
아산시           1
부산            1
울산시           1
인천            1
인천광영시         1
충천남도          1
충청남남          1
포천시           1
Name: count, dtype: int64

In [55]:
tmp = set()

In [82]:
service_set = set()
unique_cu['store_service'].apply(lambda x : [service_set.add(y) for y in x])

0              [None, None, None, None, None]
1                          [None, None, None]
2                    [None, None, None, None]
3                    [None, None, None, None]
4                    [None, None, None, None]
                         ...                 
19545                [None, None, None, None]
19546    [None, None, None, None, None, None]
19547    [None, None, None, None, None, None]
19548          [None, None, None, None, None]
19549                      [None, None, None]
Name: store_service, Length: 17633, dtype: object

In [83]:
service_set

{'bakery',
 'battery',
 'cash',
 'coffee',
 'delivery',
 'fried',
 'hour',
 'lotto',
 'mix',
 'pos',
 'toto'}

In [84]:
unique_cu['bakery'] = unique_cu.store_service.apply(lambda x : True if 'bakery' in x else False)

In [93]:
for service in service_set:
    print(service)
    unique_cu[service] = unique_cu.store_service.apply(lambda x : True if service in x else False)

cash
pos
bakery
fried
lotto
delivery
toto
battery
mix
coffee
hour


In [90]:
unique_cu.delivery.sum()

14533

In [91]:
unique_cu.value_counts('hour', normalize=True)

hour
True     0.832757
False    0.167243
Name: proportion, dtype: float64

,store_name,store_tel,store_address,store_service,sido,bakery,cash,pos,fried,lotto,delivery,toto,battery,mix,coffee,hour
0,가평군청점,0315817570,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호","[hour, delivery, fried, coffee, cash]",경기도,False,True,False,True,False,True,False,False,False,True,True
1,가평레일파크점,,경기도 가평군 가평읍 연인길 2 (읍내리),"[delivery, coffee, pos]",경기도,False,False,True,False,False,True,False,False,False,True,False
2,가평본점,,경기도 가평군 가화로 110-1 (가평읍) 1층,"[hour, delivery, coffee, cash]",경기도,False,True,False,False,False,True,False,False,False,True,True
3,가평센타점,0315817470,경기도 가평군 가평읍 오리나무길 9 1층,"[hour, delivery, coffee, cash]",경기도,False,True,False,False,False,True,False,False,False,True,True
4,가평오렌지점,,"경기도 가평군 가평읍 가화로 174, (읍내리)","[hour, delivery, coffee, cash]",경기도,False,True,False,False,False,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19545,울산학산옥교점,,울산광역시 중구 옥교3길 76 (학산동),"[hour, delivery, fried, coffee]",울산광역시,False,False,False,True,False,True,False,False,False,True,True
19546,울산학성점,0522971245,"울산광역시 중구 학성4길 65, (학성동)","[hour, delivery, bakery, fried, coffee, cash]",울산광역시,True,True,False,True,False,True,False,False,False,True,True
19547,울산학성중앙점,,울산광역시 중구 학성공원12길 20 (학성동),"[hour, delivery, bakery, fried, coffee, cash]",울산광역시,True,True,False,True,False,True,False,False,False,True,True
19548,학성동UBC점,0522431329,울산광역시 중구 구역전1길 37-9,"[hour, delivery, coffee, cash, battery]",울산광역시,False,True,False,False,False,True,False,True,False,True,True


In [104]:
# 조건이 두개이상 들어가면 비트연산 필요 
unique_cu[(unique_cu.delivery) & (unique_cu.store_address.str.find('남부순환') > -1) \
& (unique_cu.store_address.str.find('서울') > -1)]

,store_name,store_tel,store_address,store_service,sido,bakery,cash,pos,fried,lotto,delivery,toto,battery,mix,coffee,hour
5519,대치팰리스점,,서울특별시 강남구 남부순환로 2911 청우빌딩 (대치동) 청우빌딩,"[hour, delivery, fried, coffee]",서울특별시,False,False,False,True,False,True,False,False,False,True,True
5531,매봉산점,025728558,"서울특별시 강남구 남부순환로377길 6, 도곡동 (도곡동) 180동 43호","[hour, delivery, coffee, cash]",서울특별시,False,True,False,False,False,True,False,False,False,True,True
5887,공항스타점,,"서울특별시 강서구 남부순환로11길 94, (공항동)","[hour, delivery, fried, coffee]",서울특별시,False,False,False,True,False,True,False,False,False,True,True
6051,관악예술인점,025868881,서울특별시 관악구 남부순환로260길 10,"[hour, delivery, fried, coffee, cash]",서울특별시,False,True,False,True,False,True,False,False,False,True,True
6065,관악블루점,,"서울특별시 관악구 남부순환로214길 46, (봉천동) 1층","[hour, delivery, bakery, fried, coffee, cash]",서울특별시,True,True,False,True,False,True,False,False,False,True,True
6071,관악이글점,028746003,서울특별시 관악구 남부순환로 1873 대주빌딩 1층,"[hour, delivery, coffee, cash, battery]",서울특별시,False,True,False,False,False,True,False,True,False,True,True
6085,관악하정점,,서울특별시 관악구 남부순환로 1773 (봉천동) 서울대역마에스트로 캠퍼스타운 1층 ...,"[hour, delivery, coffee]",서울특별시,False,False,False,False,False,True,False,False,False,True,True
6087,관악한울점,028738885,"서울특별시 관악구 남부순환로 1801, 벽송빌딩 (봉천동)","[hour, delivery, coffee]",서울특별시,False,False,False,False,False,True,False,False,False,True,True
6104,서울대입구역점,,서울특별시 관악구 남부순환로 1833 (봉천동),"[hour, delivery, coffee]",서울특별시,False,False,False,False,False,True,False,False,False,True,True
6105,서울대파인2호점,028755989,서울특별시 관악구 남부순환로216길 24 (봉천동) 1층,"[delivery, bakery, coffee]",서울특별시,True,False,False,False,False,True,False,False,False,True,False


In [107]:
unique_cu.drop(['store_service'], axis=1, inplace=True)

KeyError: "['store_service'] not found in axis"

In [109]:
unique_cu

,store_name,store_tel,store_address,sido,bakery,cash,pos,fried,lotto,delivery,toto,battery,mix,coffee,hour
0,가평군청점,0315817570,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",경기도,False,True,False,True,False,True,False,False,False,True,True
1,가평레일파크점,,경기도 가평군 가평읍 연인길 2 (읍내리),경기도,False,False,True,False,False,True,False,False,False,True,False
2,가평본점,,경기도 가평군 가화로 110-1 (가평읍) 1층,경기도,False,True,False,False,False,True,False,False,False,True,True
3,가평센타점,0315817470,경기도 가평군 가평읍 오리나무길 9 1층,경기도,False,True,False,False,False,True,False,False,False,True,True
4,가평오렌지점,,"경기도 가평군 가평읍 가화로 174, (읍내리)",경기도,False,True,False,False,False,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19545,울산학산옥교점,,울산광역시 중구 옥교3길 76 (학산동),울산광역시,False,False,False,True,False,True,False,False,False,True,True
19546,울산학성점,0522971245,"울산광역시 중구 학성4길 65, (학성동)",울산광역시,True,True,False,True,False,True,False,False,False,True,True
19547,울산학성중앙점,,울산광역시 중구 학성공원12길 20 (학성동),울산광역시,True,True,False,True,False,True,False,False,False,True,True
19548,학성동UBC점,0522431329,울산광역시 중구 구역전1길 37-9,울산광역시,False,True,False,False,False,True,False,True,False,True,True


In [110]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 KB 18.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [117]:
import sqlalchemy
from urllib import parse

user = 'swp'
password = 'swp1'
host = '192.168.90.140'
port = 3306
database = 'encore'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [118]:
unique_cu.to_sql('cu', index=False, if_exists="append", con=engine)

17633